# FM 24 Moneyball

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import re

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [3]:
df_players_raw

,name,position,age,height,weight,inf,club,division,nat,2nd_nat,home_grown_status,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,yel,xg,starts,red,pom,pen/r,pens_s,pens_saved_ratio,pens_saved,pens_faced,pens,mins,gls/90,conc,con/90,gls,fls,fa,xg/90,xg_op,xa/90,xa,clean_sheets,av_rat,mins/gl,ast,apps,hdrs_a,tck/90,tck_w,tck_a,tck_r,shot/90,shot_%,sht/90,sht,shots_outside_box/90,shts_blckd/90,shts_blckd,shots,svt,svp,svh,sv_%,pr_passes/90,pr_passes,pres_c/90,pres_c,pres_a/90,pres_a,poss_won/90,poss_lost/90,ps_c/90,ps_c,ps_a/90,pas_a,pas_%,op_kp/90,op_kp,op_crs_c/90,op_crs_c,op_crs_a/90,op_crs_a,op_cr_%,off,gl_mst,k_tck/90,k_tck,k_ps/90,k_pas,k_hdrs/90,int/90,itc,sprints/90,hdr_%,hdrs_w/90,hdrs,hdrs_l/90,goals_outside_box,xsv_%,xgp/90,xgp,drb/90,drb,dist/90,distance,cr_c/90,cr_c,crs_a/90,cr_a,cr_c/a,conv_%,clr/90,clear,ccc,ch_c/90,blk/90,blk,asts/90,aer_a/90,rec
0,João Neves,"DM, M/AM (C)",19,174 cm,68 kg,NaN,SL Benfica,Liga Portugal Betclic,POR,NaN,-,Resolute,Level-headed,"£14,250 p/w",£40M - £56M,-,Right,4,4.88,31,2,2,-,0,-,-,-,0,2447,0.33,-,-,9,53,27,0.18,4.12,0.18,5.01,-,7.16,271.89,7,31 (4),141,2.13,58,81,72%,2.28,34%,0.77,21,1.51,0.26,7,62,-,-,-,-,7.36,200,3.60,98,11.22,305,10.11,9.08,53.70,1460,60.72,1651,88%,1.80,49,0.29,8,1.10,30,26%,0,0,0.11,3,2.10,57,0.07,1.84,50,12.14,55%,2.87,78,2.32,3,-,-,-,1.43,39,13.1km,356.0km,0.37,10,1.43,39,26%,15%,0.55,15,14,0.51,0.37,10,0.26,5.19,- - -
1,Mats Wieffer,"D (C), DM, M (C)",24,188 cm,80 kg,NaN,Feyenoord,Eredivisie,NED,NaN,-,Fairly Professional,Level-headed,"£16,500 p/w",£71M - £82M,-,Right,3,9.86,45,0,4,-,0,-,-,-,0,3457,0.29,-,-,11,27,26,0.26,1.14,0.17,6.37,-,7.21,314.27,9,45,220,2.00,77,95,81%,2.60,46%,1.20,46,1.38,0.10,4,100,-,-,-,-,6.38,245,2.71,104,10.21,392,9.35,9.55,57.48,2208,64.41,2474,89%,1.56,60,0.18,7,1.02,39,16%,6,0,0.05,2,1.67,64,0.47,1.77,68,14.40,60%,3.41,131,2.32,5,-,-,-,1.46,56,13.7km,527.4km,0.23,9,1.43,55,17%,11%,0.29,11,15,0.39,0.23,9,0.23,5.73,- - -
2,Gabriel Moscardo,DM,18,185 cm,73 kg,NaN,COR,Campeonato Brasileiro Série A Assaí,BRA,NaN,-,Fairly Determined,Level-headed,"£6,500 p/w",£15.5M - £19M,-,Right,2,1.46,13,0,2,-,0,-,-,-,0,1189,0.23,-,-,3,6,6,0.11,1.54,0.10,1.31,-,7.14,396.33,2,13 (8),52,2.20,29,41,71%,0.98,54%,0.53,7,0.38,0.15,2,13,-,-,-,-,6.21,82,2.42,32,9.46,125,8.40,7.27,67.22,888,73.20,967,92%,1.36,18,0.30,4,0.76,10,40%,0,0,0.08,1,1.36,18,0.53,1.59,21,9.16,63%,2.50,33,1.44,1,-,-,-,0.68,9,13.5km,177.7km,0.30,4,0.76,10,40%,23%,0.83,11,6,0.45,0.45,6,0.15,3.94,- - -
3,Leandro Barreiro,"DM, M (C)",24,174 cm,65 kg,Int,Mainz 05,Bundesliga,LUX,POR,-,Spirited,Evasive,"£20,500 p/w",Not for Sale,-,Right,5,1.22,31,0,0,-,0,-,-,-,0,2627,0.07,-,-,2,55,26,0.04,0.78,0.06,1.89,-,6.75,1313.50,0,31,101,2.67,78,113,69%,0.69,30%,0.21,6,0.55,0.27,8,20,-,-,-,-,3.46,101,2.57,75,11.85,346,9.66,7.43,39.88,1164,44.88,1310,89%,0.62,18,0.03,1,0.51,15,7%,1,1,0.10,3,0.62,18,0.10,2.19,64,15.79,34%,1.16,34,2.30,1,-,-,-,0.89,26,13.9km,406.1km,0.03,1,0.51,15,6%,10%,1.27,37,6,0.21,0.41,12,-,3.46,- - -
4,Xaver Schlager,"DM, M (C)",26,174 cm,76 kg,Int,RB Leipzig,Bundesliga,AUT,NaN,-,Resolute,Level-headed,"£78,000 p/w",£20M,£26.5M,Left,8,3.45,37,0,0,-,0,-,-,-,0,3060,0.06,-,-,2,47,16,0.10,-1.45,0.16,5.42,-,6.88,1530.00,5,37,84,2.29,78,108,72%,0.91,29%,0.26,9,0.68,0.03,1,31,-,-,-,-,6.38,217,2.82,96,9.97,339,7.59,6.91,45.53,1548,49.91,1697,91%,1.26,43,0.21,7,0.71,24,26%,2,0,0.03,1,1.79,61,0.06,1.53,52,15.32,37%,0.91,31,1.56,1,-,-,-,0.97,33,13.9km,472.4km,0.62,21,2.41,82,29%,6%,0.56,19,16,0.47,0.12,4,0.15,2.47,- - -
5,Albert Sambi Lokonga,"DM, M (C)",24,183 cm,74 kg,Int,Arsenal,Premier League,BEL,COD,-,Fairly Determined,Level-headed,"£50,000 p/w",£30M,£30M,Right,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,- - -
6,Jesús Castillo,"DM, M (C)",22,185 cm,80 kg,NaN,Gil Vicente,Liga Portugal Betclic,PER,NaN,-,Balanced,

In [4]:
### Read raw data

# Select file path for raw data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM 24 exports\Summer 2024\Defensive Midfielders.html"

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM Data Lab outputs\Summer 2024\Defensive Midfielders SL.csv"

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [ ]:
df_players.head()

In [ ]:
# # Keep only natural DMs
# path = 'data/Natural DMs.html'

# # Ingest in a dataframe
# df_dms = pd.read_html(path, encoding='utf-8')[0]

# # Format column names 
# df_dms.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_dms.columns]

# df_dms['natural_dm'] = 'Yes'

# df_dms = df_dms[['uid', 'natural_dm']]

# df_players = df_players.merge(df_dms, 
#                               left_on='id',
#                               right_on='uid',
#                               how='left')

# df_players = df_players[df_players['natural_dm'] == 'Yes']

In [5]:
# Get transfer valuation estimation
def parse_transfer_value(val):
    if pd.isnull(val):
        return None

    # Remove dollar signs and spaces
    val = val.replace('£', '').replace(' ', '')

    # If it's a range like "850K-8.4M"
    if '-' in val:
        low_str, high_str = val.split('-')
        low = parse_single_value(low_str)
        high = parse_single_value(high_str)
        if low is not None and high is not None:
            return (low + high) * (6/10)
    else:
        return parse_single_value(val)

    return None  # fallback

def parse_single_value(s):
    try:
        if s.endswith('M'):
            return float(s[:-1]) * 1_000_000
        elif s.endswith('K'):
            return float(s[:-1]) * 1_000
        else:
            return float(s)  # just in case it's a plain number
    except:
        return None

# Apply to your DataFrame
df_players['price_estimate_M'] = df_players['transfer_value'].apply(parse_transfer_value) / 1000000

## Plot function

In [6]:
def scatter_plot(df, x_metric, y_metric, z_metric, 
                 title, show_color_bar=False, show_name_label=True): 
    # Set text to 'name' if show_name_label is True, else empty string
    text_col = 'name' if show_name_label else None

    # Create plotly figure
    fig = px.scatter(
        df,
        x=x_metric,
        y=y_metric,
        color=z_metric,
        text=text_col,
        color_continuous_scale='RdYlGn',
        labels={z_metric: f'Color Scale ({z_metric})'},
        title=title,
        width=1200, 
        height=700    
    )

    # Format traces
    fig.update_traces(
        marker=dict(size=10), 
        textposition='top center',
        textfont=dict(size=10),
        customdata=df[['name', 'club']],  # Always pass 'name' for hovertemplate
        hovertemplate=(
            "<b>%{customdata[0]}</b><br>" +
            "<b>%{customdata[1]}</b><br><br>" +
            f"{x_metric}: " + "%{x}<br>" +
            f"{y_metric}: " + "%{y}<br>" +
            f"{z_metric}: " + "%{marker.color}<extra></extra>"
        )
    )

    fig.update_layout(
        title={
            'text': f'{title} ({len(df): ,.0f})',
            'x': 0.5,
            'xanchor': 'center'
        },
        coloraxis_colorbar=dict(title=z_metric),
        coloraxis_showscale=show_color_bar,  # Show or hide the color bar
        plot_bgcolor='darkgray',
        paper_bgcolor='darkgray',
        xaxis=dict(showgrid=True, gridcolor='white', zeroline=False),
        yaxis=dict(showgrid=True, gridcolor='white', zeroline=False)
    )

    fig.show()


## Passing analysis

### Full universe

In [8]:
# Initialize list of targets
targets = df_players['id']

scatter_plot(df_players,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=True
)

In [11]:
# Filter data 
mask = (
        # (df_players['passes_attempted/90'] >= 45) & \
        # (df_players['progressive_pass_rate'] >= 7) & \
        # (df_players['pass_completion_%'] >= 85) & \
        # (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Possession Analysis

In [13]:
# Full universe 
scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=True
)

In [ ]:
# Filter data 
mask = (
        (df_players['possession_lost/90'] <= 8) & \
        (df_players['possession_won/90'] >= 7.5) & \
        (df_players['poss+_/90'] >= 0) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Aggresion vs Control
Identify aggresive and foul prones targets

In [15]:
# Full universe 
scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=True
)

In [16]:
# Compute tackes/fouls ratio
df_players['tackles_comp-fouls/90'] = df_players['tackles_completed/90'] / df_players['fouls_committed/90']
# Filter data 
mask = (
        # (df_players['tackles_comp-fouls/90'] <= 2) & \
        # (df_players['tackles_completed/90'] >= 1.5) & \
        # (df_players['tackle_completion_%'] >= 65) & \
        #(df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Pressure quality

In [17]:
# Full universe 
scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

In [ ]:
# Filter data 
mask = (
        (df_players['pressures_completed/90'] >= 2.0) & \
        (df_players['pressure_success_%'] >= 20) & \
        (df_players['duels_win_%'] >= 50) & \
        (df_players['id'].isin(targets)) | \
        (df_players['name'].str.contains('Barreiro')) 
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Presence

In [18]:
# Full universe 
scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
) 

In [ ]:
df_players[df_players['name'] == 'Casemiro']

In [ ]:
# Filter data 
mask = (
        (df_players['interceptions/90'] >= 1.4) & \
        (df_players['blocks/90'] >= 0.25) & \
        (df_players['clearances/90'] >= 0.5) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
)

#df_targets = df_players[mask]

## Distance

In [19]:
# Full universe 
scatter_plot(df_targets,
             x_metric='distance_covered/90', 
             y_metric='defensive_actions/90',
             z_metric='creative_actions/90',
             title='Distance', 
             show_name_label=True
) 

In [ ]:
df_targets.groupby('division')['id'].count()

In [ ]:
df_targets.groupby('division').get_group('Sky Bet Championship').sort_values('age')